## Query header

In this notebook, we show several ways to query the fits headers with Gen3\
Craig Lage - 03-Jun-21

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.bestEffort import BestEffortIsr
from lsst.summit.utils.plotting import plot
from lsst.summit.utils import quickSmooth
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

In [ ]:
butler = butlerUtils.makeDefaultLatissButler(embargo=False)

In [ ]:
butler.collections

In [ ]:
butler.registry.queryDatasetTypes()

In [ ]:
dayObs = 20240313
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter, record.target_name)


In [ ]:
expId = 2024031300003
mData = butler.get('raw.metadata', detector=0, exposure=expId)
for key in mData.keys():
    print(key, mData[key])

In [ ]:
dataId = dict(day_obs=20240308, detector=0, seq_num=199)
bestEffort = BestEffortIsr(embargo=False)
bestEffort.getExposure(dataId)
dataType = 'quickLookExp'
exp = butler.get(dataType, dataId)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,10))
plot(exp, figure=fig, stretch='ccs')
[ax1, ax2] = fig.axes
ax1.set_title(f"AuxTel, 20240308 seqNum 199", fontsize = 24)
slices = [[1830, 0.45, 0.6], [500, 0.85, 1.0], [3500, 0.1, 0.25]]
for [y, xmin, xmax] in slices:
    ax1.axhline(y, xmin, xmax, color='yellow')
plt.savefig("/home/cslage/DATA/Defects_1_20240308_199.png")

In [ ]:
mask = exp.getMaskedImage().getArrays()[1]
fig, axs = plt.subplots(2,1,figsize = (5,5))
axs[0].plot(exp.image.array[1830, 2130:2180], marker='x')
axs[1].plot(mask[1830, 2130:2180], marker='x')

In [ ]:
len(im.getArrays())

In [ ]:
plt.imshow(mask[1700:2100, 1950:2350 ])
plt.colorbar()

In [ ]:
plt.plot(mask[1830, 2130:2180], marker='x')

In [ ]:
im = exp.getMaskedImage()
fig = plt.figure(figsize=(10,10))
_ = plot(im[1950:2350, 1700:2100], figure=fig, stretch='ccs')
[ax1, ax2] = fig.axes
ax1.imshow(mask.array[1700:2100, 1950:2350 ])

In [ ]:
from lsst.summit.utils import quickSmooth
from mpl_toolkits.axes_grid1 import make_axes_locatable
def plotExp(exposure):
    # Borrowed from summit utils
    data = quickSmooth(exposure.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    ax1.set_title(f"{expId}")
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure, vmin, vmax


In [ ]:
plotExp(be)

In [ ]:
%matplotlib inline
fig = plotExp(exp)
plt.suptitle(f" quickLookExp {expId}")
#fig.savefig(f"/home/c/cslage/u/AuxTel/isr/before_after/Before.png")

In [ ]:
expId=2024022200254
mine = butler.get("postISRCCD", instrument="LATISS", detector=0, exposure=expId, collections='u/czw/tests/t.20240222a')

In [ ]:
fig = plotExp(mine)
plt.suptitle(f"Chris' run with full ISR {expId}")

In [ ]:
for i in range(50):
    y = int(1750 + 2 * i)
    plt.plot(exp.image.array[y, 2000:2400])
    print(y, np.max(exp.image.array[y, 2000:2400]))

In [ ]:
plt.title(f"Chris' full ISR, slice at Y=1820  {expId}")
plt.plot(mine.image.array[1820, :], marker='x')
plt.xlim(2100,2200)
plt.xlabel("X (pixels)")
plt.ylabel("Signal(ADU)")

In [ ]:
plt.title(f"Chris' full ISR, slice at Y=1820  {expId}")
plt.plot(mine.image.array[1820, :], marker='x')
plt.xlim(2145, 2152)
plt.xlabel("X (pixels)")
plt.ylabel("Signal(ADU)")